In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [5]:
class VRPDP_ACO:
    def __init__(self, alpha=1, beta=2, rho=0.5, Q=100, num_ants=5, num_iterations=100):
        # ACO algorithm parameters
        self.alpha = alpha              # Influence of pheromone
        self.beta = beta                # Influence of heuristic (1 / travel time)
        self.rho = rho                  # Evaporation rate
        self.Q = Q                      # Pheromone deposit factor
        self.num_ants = num_ants
        self.num_iterations = num_iterations

    def _compute_route_distance(self, route, distance_matrix, customer_index):
        """Compute total distance of a given route based on heuristic matrix."""
        total_distance = 0
        for i in range(len(route) - 1):
            from_node = route[i][0]
            to_node = route[i][1]
            from_idx = customer_index[from_node]
            to_idx = customer_index[to_node]
            total_distance += distance_matrix[from_idx][to_idx]
        return total_distance

    def _compute_distance_matrix(self, all_nodes, travel_times):
        """Compute and return the heuristic matrix (1 / travel time) between all pairs of nodes."""
        n = len(all_nodes)
        distance_matrix = np.zeros((n, n))
        index_map = {node: idx for idx, node in enumerate(all_nodes)}

        # Only compute for (i ≠ j) once per pair, use symmetry if available
        for i, from_node in enumerate(all_nodes):
            for j, to_node in enumerate(all_nodes):
                if i == j:
                    continue
                if (from_node, to_node) in travel_times:
                    travel_time = travel_times[(from_node, to_node)]
                elif (to_node, from_node) in travel_times:
                    travel_time = travel_times[(to_node, from_node)]
                else:
                    travel_time = float('inf')  # If missing, assume unreachable
                distance_matrix[i][j] = 1 / travel_time
        return distance_matrix

    def _choose_next_customer(self, current, candidates, pheromone_matrix, distance_matrix, customer_index):
        """
        Select the next customer using probabilistic transition rules.

        probabilities[i] ∝ pheromone^alpha * heuristic^beta
        where:
          - pheromone = desirability based on past experience
          - heuristic = 1 / travel time = closeness
        """
        probabilities = np.zeros(len(candidates))
        for i, cust in enumerate(candidates):
            i_from = customer_index[current]
            i_to = customer_index[cust]
            pheromone = pheromone_matrix[i_from][i_to]
            heuristic = distance_matrix[i_from][i_to]
            probabilities[i] = (pheromone ** self.alpha) * (heuristic ** self.beta)
        probabilities /= probabilities.sum()
        return np.random.choice(candidates, p=probabilities)

    def solve(self, depot, customers, vehicles, travel_times):
        # ============================
        # Initialization of Structures
        # ============================

        all_nodes = ['Depot'] + list(customers.keys())
        customer_index = {customer: idx for idx, customer in enumerate(all_nodes)}
        n = len(all_nodes)

        # Initialize pheromone and heuristic matrices
        pheromone_matrix = np.ones((n, n))
        distance_matrix = self._compute_distance_matrix(all_nodes, travel_times)

        best_route = None
        best_distance = float('inf')

        # ====================================
        # Main ACO Optimization Loop
        # ====================================

        for iteration in range(self.num_iterations):
            ant_routes = []

            for _ in range(self.num_ants):
                route = []
                current_location = 'Depot'
                capacity_remaining = vehicles['1']['capacity']
                unvisited_customers = list(customers.keys())

                while unvisited_customers:
                    feasible_customers = [c for c in unvisited_customers if customers[c]['demand'] <= capacity_remaining]
                    if not feasible_customers:
                        break

                    next_customer = self._choose_next_customer(
                        current_location,
                        feasible_customers,
                        pheromone_matrix,
                        distance_matrix,
                        customer_index
                    )
                    route.append((current_location, next_customer))
                    current_location = next_customer
                    capacity_remaining -= customers[next_customer]['demand']
                    unvisited_customers.remove(next_customer)

                # Return to depot
                route.append((current_location, 'Depot'))
                ant_routes.append(route)

            # Pheromone evaporation
            pheromone_matrix *= (1 - self.rho)

            # Pheromone update based on ant solutions
            for route in ant_routes:
                route_distance = self._compute_route_distance(route, distance_matrix, customer_index)
                deposit = self.Q / route_distance
                for i in range(len(route) - 1):
                    from_idx = customer_index[route[i][0]]
                    to_idx = customer_index[route[i][1]]
                    pheromone_matrix[from_idx][to_idx] += deposit

                if route_distance < best_distance:
                    best_distance = route_distance
                    best_route = route
        return best_route

In [ ]:
# ================
# Sample Usage
# ================

depot = (40.7128, -74.0060)
customers = {
    'A': {'location': (40.7291, -74.0113), 'demand': 5, 'time_window': (9, 12)},
    'B': {'location': (40.7429, -73.9922), 'demand': 3, 'time_window': (10.5, 13.5)},
    'C': {'location': (40.7180, -73.9962), 'demand': 2, 'time_window': (13, 16)},
}
vehicles = {
    '1': {'capacity': 10, 'location': depot},
    '2': {'capacity': 8, 'location': depot},
}
travel_times = {
    ('Depot', 'A'): 15, ('Depot', 'B'): 20, ('Depot', 'C'): 10,
    ('A', 'Depot'): 15, ('A', 'B'): 25, ('A', 'C'): 18,
    ('B', 'Depot'): 20, ('B', 'A'): 25, ('B', 'C'): 12,
    ('C', 'Depot'): 10, ('C', 'A'): 18, ('C', 'B'): 12,
}

aco_solver = VRPDP_ACO()
best_route = aco_solver.solve(depot, customers, vehicles, travel_times)

print("Best Route:")
for step in best_route:
    print(step)